In [1]:
import os
import numpy as np
from functools import partial
from transformers import (
    AutoImageProcessor, 
    AutoModelForImageClassification, 
    TrainingArguments, 
    Trainer
)
from datasets import load_dataset
import torchvision.transforms as T
from PIL import Image

2025-02-28 16:05:38.931748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-28 16:05:39.793084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-12.6/lib64
2025-02-28 16:05:39.793208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7

In [2]:
# --------------------------
# 1. Load the Pre-trained Model and Processor
# --------------------------
processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression")
model = AutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

In [3]:
# --------------------------
# 2. Load Your Dataset
# --------------------------
# For this example, we're using an imagefolder dataset. Ensure your dataset is organized with subfolders for each label.
# Replace '/path/to/your/dataset' with the actual path (e.g., JAFFE or a curated folder)
dataset = load_dataset("imagefolder", data_dir="/home/natalyagrokh/img_datasets/combo_ferckja_dataset", split="train")

Resolving data files:   0%|          | 0/37081 [00:00<?, ?it/s]

In [5]:
# --------------------------
# 3. Define mapping: dataset label -> pre-trained model label
# --------------------------
# Updated mapping using lowercase keys
label_mapping = {
    'anger': 'Angry',
    'contempt': 'Disgust',  # Merge "contempt" with "disgust"
    'disgust': 'Disgust',
    'fear': 'Fear',
    'happiness': 'Happy',
    'sadness': 'Sad',
    'surprise': 'Surprise',
    'neutral': 'Neutral'
}

# Numerical mapping for the pre-trained model's labels.
num_mapping = {
    'Angry': 0,
    'Disgust': 1,
    'Fear': 2,
    'Happy': 3,
    'Sad': 4,
    'Surprise': 5,
    'Neutral': 6
}

def reconcile_labels(example):
    # If the label is already an integer, convert it to a string using the dataset features.
    if isinstance(example["label"], int):
        # Use dataset.features["label"].int2str to get the string label.
        original_label = dataset.features["label"].int2str(example["label"]).strip().lower()
    else:
        original_label = example["label"].strip().lower()
    
    # Map the lowercased label to the pre-trained model's expected label.
    pretrain_label = label_mapping.get(original_label)
    
    if pretrain_label is None:
        # If not recognized, mark it for filtering.
        example["label"] = -1
    else:
        # Convert the mapped label to its corresponding integer.
        example["label"] = num_mapping[pretrain_label]
    return example

# Apply the reconciliation function to the dataset.
dataset = dataset.map(reconcile_labels)

# Filter out any examples that were marked as unrecognized.
dataset = dataset.filter(lambda x: x["label"] != -1)

Map:   0%|          | 0/37081 [00:00<?, ? examples/s]

Filter:   0%|          | 0/37081 [00:00<?, ? examples/s]

In [6]:
print("Total examples after filtering:", len(dataset))

Total examples after filtering: 37081


In [8]:
# --------------------------
# 4. Define Data Augmentation and Preprocessing Transformation
# --------------------------
# Use torchvision transforms for lightweight CPU-based augmentation.
data_augment = T.Compose([
    T.RandomHorizontalFlip(),                # Random horizontal flip
    T.RandomRotation(10),                      # Random rotation within ±10 degrees
    T.ColorJitter(brightness=0.1, contrast=0.1)  # Slight brightness and contrast changes
])

def transform_function(example, processor):
    # Ensure the image is loaded as a PIL image.
    if not isinstance(example["image"], Image.Image):
        example["image"] = Image.open(example["image"])
    
    # Convert image to RGB mode if it isn't already.
    if example["image"].mode != "RGB":
        example["image"] = example["image"].convert("RGB")
    
    # Apply data augmentation.
    augmented_image = data_augment(example["image"])
    
    # Process the augmented image using the pre-trained processor.
    inputs = processor(augmented_image, return_tensors="pt")
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    
    # Add the label (ensure the label is in the proper format, e.g. integer).
    inputs["labels"] = example["label"]
    return inputs

# Map the transformation to every example in the dataset.
dataset = dataset.map(partial(transform_function, processor=processor))

Map:   0%|          | 0/37081 [00:00<?, ? examples/s]

In [9]:
# --------------------------
# 5. Split Dataset into Training and Validation Sets
# --------------------------
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [10]:
# --------------------------
# 6. Define Training Arguments for Robust Fine-Tuning
# --------------------------
training_args = TrainingArguments(
    output_dir="./finetuned_vit_model",    # Directory to save checkpoints and the final model
    evaluation_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save checkpoint at each epoch
    learning_rate=1e-4,                    # A conservative learning rate for fine-tuning
    per_device_train_batch_size=8,         # Adjust based on your CPU memory limits
    per_device_eval_batch_size=8,
    num_train_epochs=5,                    # Fine-tune for a few epochs (adjust as needed)
    load_best_model_at_end=True,           # Automatically load the best model when training finishes
    metric_for_best_model="accuracy",      # Monitor accuracy for best model selection
    logging_dir="./logs",                  # Directory for TensorBoard logs
)

/opt/conda/envs/ml_expressions/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# --------------------------
# 7. Define a Compute Metrics Function for Evaluation
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

In [12]:
# --------------------------
# 8. Initialize and Run the Trainer for Fine-Tuning
# --------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.853900,0.828130,0.696238
2,0.581600,0.836145,0.710260
3,0.302200,0.954819,0.732641
4,0.109100,1.296146,0.741809
